In [1]:
request_header = <<
    0xAA, 0xAA, # ID
    0x01, 0x00, # Query parameters
    0x00, 0x01, # Number of questions
    0x00, 0x00, # Number of answers
    0x00, 0x00, # Number of authority records
    0x00, 0x00  # Number of additional records
>>

<<170, 170, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0>>

In [2]:
byte_size(request_header) === 2 * 6

true

In [3]:
question = <<
    0x07, 0x65, # 'example' has length 7, e
    0x78, 0x61, # x, a
    0x6D, 0x70, # m, p
    0x6C, 0x65, # l, e
    0x03, 0x63, # 'com' has length 3, c
    0x6F, 0x6D, # o, m
    0x00,       # zero byte to end the QNAME 
    0x00, 0x01, # QTYPE 
    0x00, 0x01  # QCLASS
>>

<<7, 101, 120, 97, 109, 112, 108, 101, 3, 99, 111, 109, 0, 0, 1, 0, 1>>

In [4]:
byte_size(question) === 17

true

In [5]:
request = request_header <> question

<<170, 170, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 7, 101, 120, 97, 109, 112, 108, 101, 3, 99, 111, 109, 0, 0, 1, 0, 1>>

In [6]:
alias :gen_udp, as: UDP
passive = [active: false]
binary = [:binary]

{:ok, s} = UDP.open(1024)
:ok = :inet.setopts(s, passive ++ binary)
:ok = UDP.send(s, '8.8.8.8', 53, request)
{:ok, r} = UDP.recv(s, 8)
UDP.close(s)

:ok

In [7]:
{_, _, x} = r; x

<<170, 170, 129, 128, 0, 1, 0, 1, 0, 0, 0, 0, 7, 101, 120, 97, 109, 112, 108, 101, 3, 99, 111, 109, 0, 0, 1, 0, 1, 192, 12, 0, 1, 0, 1, 0, 0, 43, 203, 0, 4, 93, 184, 216, 34>>

In [8]:
response_header = <<
    0xAA, 0xAA, # Same ID as before
    0x81, 0x80, # Different flags, we'll look at this below
    0x00, 0x01, # 1 question
    0x00, 0x01, # 1 answer
    0x00, 0x00, # No authority records
    0x00, 0x00  # No additional records
>>

<<170, 170, 129, 128, 0, 1, 0, 1, 0, 0, 0, 0>>

In [9]:
<< i::binary-12, j::binary-17, k::binary >> = x

<<170, 170, 129, 128, 0, 1, 0, 1, 0, 0, 0, 0, 7, 101, 120, 97, 109, 112, 108, 101, 3, 99, 111, 109, 0, 0, 1, 0, 1, 192, 12, 0, 1, 0, 1, 0, 0, 43, 203, 0, 4, 93, 184, 216, 34>>

In [10]:
i == response_header

true

In [11]:
j == question

true

In [12]:
answer = <<
    0xC0, 0x0C, # NAME
    0x00, 0x01, # TYPE
    0x00, 0x01, # CLASS
    0x00, 0x00,
    0x18, 0x4C, # TTL
    0x00, 0x04, # RDLENGTH = 4 bytes
    0x5D, 0xB8,
    0xD8, 0x22 # RDDATA
>>

<<192, 12, 0, 1, 0, 1, 0, 0, 24, 76, 0, 4, 93, 184, 216, 34>>

In [13]:
k == answer

false

In [14]:
k

<<192, 12, 0, 1, 0, 1, 0, 0, 43, 203, 0, 4, 93, 184, 216, 34>>

In [15]:
String.myers_difference k, answer

[eq: <<192, 12, 0, 1, 0, 1, 0, 0>>, del: <<43, 203>>, ins: <<24, 76>>, eq: <<0, 4, 93, 184, 216, 34>>]